In [1]:
%tensorflow_version 1.x
#!git clone https://github.com/tensorflow/cleverhans.git
!pip install cleverhans
!pip install adversarial-robustness-toolbox

TensorFlow 1.x selected.
     |████████████████████████████████| 204kB 2.8MB/s 
     |████████████████████████████████| 163kB 47.4MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 491kB 2.8MB/s 
     |████████████████████████████████| 7.1MB 8.3MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [25]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, AveragePooling1D, Flatten, Reshape, TimeDistributed
from keras.layers import LSTM
from keras.datasets import imdb
from keras.utils import to_categorical

max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...
25000 train sequences
25000 test sequences
x_train shape: (25000,)
x_test shape: (25000,)


In [0]:
word_to_index = imdb.get_word_index()
index_to_word = {v: k for k, v in word_to_index.items()}

In [27]:
print(x_train[0])
print(" ".join([index_to_word[x] for x in x_train[0]]))
print("Min value: ", min(y_train), "Max value: ", max(y_train) )

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
the as you with out themselves powerful lets lov

In [28]:
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in x_train[0]])
print(decoded_review)

? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what th

In [29]:
import numpy as np
average_length = np.mean([len(x) for x in x_train])
median_length = sorted([len(x) for x in x_train])[len(x_train) // 2]

print("Average sequence length: ", average_length)
print("Median sequence length: ", median_length)

max_sequence_length = 80

Average sequence length:  238.71364
Median sequence length:  178


In [30]:
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)
print(y_train[0])
print(y_test[0])

Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)
y_train shape: (25000,)
y_test shape: (25000,)
1
0


In [31]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)
print(y_train[0])
print(y_test[0])

y_train shape: (25000, 2)
y_test shape: (25000, 2)
[0. 1.]
[1. 0.]


In [32]:
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in x_train[0]])
print(decoded_review)

that played the ? of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you think the whole story was so lovely because it was true and was someone's life after all that was shared with us all


In [33]:
from keras.layers import  AveragePooling2D

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 80, input_length=80,mask_zero=True))
model.add(LSTM(80, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
# model.add(AveragePooling2D(pool_size=4, strides=None, padding='valid', data_format='channels_last'))
# AvgPoolinD3

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Build model...
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 80, 80)            1600000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 80)                51520     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 162       
Total params: 1,651,682
Trainable params: 1,651,682
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
print('Train...')
model.fit(x_train, y_train, batch_size=batch_size, epochs=15, validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Train...

Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 88s 4ms/step - loss: 0.4581 - accuracy: 0.7822 - val_loss: 0.4029 - val_accuracy: 0.8228
Epoch 2/15
25000/25000 [==============================] - 88s 4ms/step - loss: 0.3017 - accuracy: 0.8786 - val_loss: 0.3770 - val_accuracy: 0.8389
Epoch 3/15
25000/25000 [==============================] - 87s 3ms/step - loss: 0.2319 - accuracy: 0.9090 - val_loss: 0.4090 - val_accuracy: 0.8300
Epoch 4/15
25000/25000 [==============================] - 87s 3ms/step - loss: 0.1676 - accuracy: 0.9370 - val_loss: 0.4440 - val_accuracy: 0.8278
Epoch 5/15
25000/25000 [==============================] - 87s 3ms/step - loss: 0.1381 - accuracy: 0.9484 - val_loss: 0.5299 - val_accuracy: 0.8220
Epoch 6/15
25000/25000 [==============================] - 87s 3ms/step - loss: 0.1016 - accuracy: 0.9645 - val_loss: 0.6075 - val_accuracy: 0.8207
Epoch 7/15
25000/25000 [==============================] - 

In [0]:
model.save("Colab_LSTM_Model.h5")

In [35]:
# Creating Attack
#Preparing Train and Test Subet set
from sklearn.model_selection import train_test_split

x_train_sub, _,y_train_sub,_ = train_test_split(x_train, y_train, stratify=y_train, train_size=.08, shuffle = True,)
x_test_sub, _, y_test_sub,_ = train_test_split(x_test, y_test, stratify=y_train, train_size=.004, shuffle = True,)

print('Training set size:',x_train_sub.shape)
print('Test set size',x_test_sub.shape)


def jacobianFun(x):
  """
  Calculate jacobian of logits wrt input.
  """
  x = tf.convert_to_tensor(x,dtype=tf.float32)
  x = tf.expand_dims(x,0)

  with tf.GradientTape(watch_accessed_variables=False, persistent=True) as tape:
    tape.watch(e.variables)
    y = model(x)

  return tf.squeeze(tape.jacobian(y, e.variables, parallel_iterations=100, experimental_use_pfor=False))
  

Training set size: (2000, 80)
Test set size (100, 80)


In [0]:
# Adversarial Attack
def imdb_attack(f, x, D=np.array(list(word_index.values()),dtype='int64'), max_iter=100):
  y = f.predict(x)
  y = 0 if (y[0] <= 0.5) else 1

  x_adversarial = x.copy()

  ad_prediction = convert_to_label(f.predict(x_adversarial))

  jacobian = jacobianFun(np.squeeze(x))

  iteration = 0

  while ad_prediction == y:

    iteration += 1

    if iteration > max_iter:
      break

    i = np.random.randint(0,80,1)[0]

    j_x = jacobian[i]

    z = tf.argmin(np.sign(D - x_adversarial[0][i])) #+ 1 #b/c dictionary starts on index 1

    w = np.abs(z - np.sign(j_x[i]))

    if w==None: break 

    if not w < max_features:
      continue
    
    x_adversarial[0][i]=w

    ad_prediction = convert_to_label(f.predict(x_adversarial))


  if ad_prediction != y:
    print('Adversarial EXAMPLE Created in ',iteration,'Iterations')

  return x_adversarial

In [40]:
#creting adversarial examples
adversarial_example = np.array([imdb_attack(model,np.expand_dims(i,0)) for i in x_test_sub]).squeeze()

ValueError: ignored

In [0]:
#Changes in sentences
for i in range(len(adversarial_example)):
  print('Sentence ',i,'Original')
  show_sentence(x_test_sub[i])
  print('\nSentence ',i,'Adversarial')
  show_sentence(adversarial_example[i])
  print('----------------------------------------------------------\n')

In [0]:
#testing model on adversarial examples
print(model.evaluate(x_test_sub,y_test_sub,batch_size=batch_size))
print(model.evaluate(adversarial_example, y_test_sub, batch_size=batch_size)) 